In [37]:
import numpy as np
from pytket.circuit import Circuit, Op, OpType

import sys

sys.path.append('../')
from math import pi
from canopus import utils
import ashn.utils
from scipy.stats import unitary_group
import ashn

In [38]:
sqisw_coord = np.array([0.25, 0.25, 0]) * pi / 2
utils.determine_ashn_gate_duration(*sqisw_coord, 1, 1, 0)

0.7853981633974483

In [39]:
utils.determine_ashn_gate_duration(pi / 4, 0, 0, 1, 1, 0)

1.5707963267948966

In [40]:
RAND_SU4_CACHE = []
np.random.seed(123)
for _ in range(100000):
    x, y, z = ashn.utils.canonical_decompose(unitary_group.rvs(4))[-1]
    RAND_SU4_CACHE.append((x, y, z))


In [41]:
RAND_ABC_CACHE = []
np.random.seed(123)
for _ in range(100000):
    RAND_ABC_CACHE.append(ashn.utils.normalize_hamiltonian(ashn.utils.random_hermitian(4))[0])


In [42]:
def average_gate_duration(a, b, c):
    durations = []
    for x, y, z in RAND_SU4_CACHE:
        durations.append(ashn.determine_gate_duration(x, y, z, a, b, c))
    return np.mean(durations) * np.linalg.norm([a, b, c], ord=1)


def cnot_duration(a, b, c):
    return ashn.determine_gate_duration(pi / 4, 0, 0, a, b, c) * np.linalg.norm([a, b, c], ord=1)


def iswap_duration(a, b, c):
    return ashn.determine_gate_duration(pi / 4, pi / 4, 0, a, b, c) * np.linalg.norm([a, b, c], ord=1)


def sqisw_duration(a, b, c):
    return ashn.determine_gate_duration(pi / 8, pi / 8, 0, a, b, c) * np.linalg.norm([a, b, c], ord=1)


def b_duration(a, b, c):
    return ashn.determine_gate_duration(pi / 4, pi / 8, 0, a, b, c) * np.linalg.norm([a, b, c], ord=1)


def swap_duration(a, b, c):
    return ashn.determine_gate_duration(pi / 4, pi / 4, pi / 4, a, b, c) * np.linalg.norm([a, b, c], ord=1)

In [44]:
print(cnot_duration(1, 0, 0) / pi)
print(iswap_duration(1, 0, 0) / pi)
print(sqisw_duration(1, 0, 0) / pi)
print(b_duration(1, 0, 0) / pi)
print(swap_duration(1, 0, 0) / pi)

0.25
0.5
0.25
0.375
0.75


In [49]:
print(np.mean([cnot_duration(a,b,c) for a,b,c in RAND_ABC_CACHE]))

1.22804282981187


In [56]:
random_abc_durations = {
    # 'SU(4)': np.mean([average_gate_duration(a,b,c) for a,b,c in RAND_ABC_CACHE]),
    'CNOT': np.mean([cnot_duration(a,b,c) for a,b,c in RAND_ABC_CACHE]),
    'iSWAP': np.mean([iswap_duration(a,b,c) for a,b,c in RAND_ABC_CACHE]),
    'SQiSW': np.mean([sqisw_duration(a,b,c) for a,b,c in RAND_ABC_CACHE]),
    'B': np.mean([b_duration(a,b,c) for a,b,c in RAND_ABC_CACHE]),
    'SWAP': np.mean([swap_duration(a,b,c) for a,b,c in RAND_ABC_CACHE]),
}


In [57]:
random_abc_durations

{'CNOT': 1.22804282981187,
 'iSWAP': 1.8976590834308076,
 'SQiSW': 0.9488295417154038,
 'B': 1.4346683092344887,
 'SWAP': 2.3561944901923444}

1.8976590834308076

In [33]:
np.mean([average_gate_duration(a,b,c) for a,b,c in RAND_ABC_CACHE])

KeyboardInterrupt: 

In [34]:
def determine_gate_duration(x, y, z, a, b, c):
    tau0 = x / a
    tau_plus = (x + y - z) / (a + b - c)
    tau_minus = (x + y + z) / (a + b + c)
    tau1 = max(tau0, tau_plus, tau_minus)

    tau0_prime = (np.pi / 2 - x) / a
    tau_plus_prime = (np.pi / 2 - x + y + z) / (a + b - c)
    tau_minus_prime = (np.pi / 2 - x + y - z) / (a + b + c)
    tau2 = max(tau0_prime, tau_plus_prime, tau_minus_prime)

    tau = min(tau1, tau2)
    return tau

durations = []

for a, b, c in RAND_ABC_CACHE: # RAND_ABC_CACHE is a list consisting of 10000 (a,b,c) tuples
    durations_part = []
    for x, y, z in RAND_SU4_CACHE: # RAND_SU4_CACHE is a list consisting of 100000 (x,y,z) tuples
        durations_part.append(determine_gate_duration(x, y, z, a, b, c))
    durations.append(np.mean(durations_part) * np.linalg.norm([a, b, c], ord=1))
avg_duration = np.mean(durations)

0.613, 0.244, 0.046 -> 0.396
0.580, 0.058, -0.018 -> 0.385
0.427, 0.366, 0.100 -> 0.465
0.513, 0.291, 0.097 -> 0.433
0.506, 0.274, -0.039 -> 0.399
0.559, 0.315, -0.076 -> 0.416
0.499, 0.355, 0.114 -> 0.452
0.502, 0.149, 0.036 -> 0.396
0.529, 0.285, 0.147 -> 0.470
0.519, 0.098, -0.009 -> 0.380
0.626, 0.229, -0.066 -> 0.405
0.389, 0.251, -0.200 -> 0.608
0.621, 0.261, -0.039 -> 0.393
0.876, 0.286, -0.077 -> 0.401
0.661, 0.178, 0.093 -> 0.421
0.638, 0.364, -0.013 -> 0.386
0.488, 0.208, 0.061 -> 0.411
0.483, 0.341, 0.026 -> 0.402
0.858, 0.275, 0.129 -> 0.422
0.576, 0.298, -0.002 -> 0.380
0.556, 0.168, -0.049 -> 0.401
0.417, 0.359, -0.050 -> 0.433
0.467, 0.068, 0.041 -> 0.404
0.627, 0.185, 0.021 -> 0.385
0.545, 0.486, -0.017 -> 0.417
0.689, 0.146, 0.008 -> 0.379
0.699, 0.169, 0.007 -> 0.378
0.354, 0.164, -0.033 -> 0.402
0.416, 0.130, 0.009 -> 0.381
0.392, 0.334, -0.064 -> 0.444


KeyboardInterrupt: 

In [9]:
import torch

In [31]:
rand_abc_cache = torch.from_numpy(np.asarray(RAND_ABC_CACHE))
rand_su4_cache = torch.from_numpy(np.asarray(RAND_SU4_CACHE))

In [32]:

def determine_gate_duration_vectorized(xyz, abc):
    """向量化版本，接受shape为(N, M)的输入"""
    # 展开输入维度
    x = xyz[..., 0]  # shape: (N, M)
    y = xyz[..., 1]
    z = xyz[..., 2]
    a = abc[..., 0]  # shape: (N, M)
    b = abc[..., 1]
    c = abc[..., 2]
    # 第一阶段计算
    tau0 = x / a
    tau_plus = (x + y - z) / (a + b - c)
    tau_minus = (x + y + z) / (a + b + c)
    tau1 = torch.maximum(torch.maximum(tau0, tau_plus), tau_minus)
    # 第二阶段计算
    tau0_prime = (torch.pi/2 - x) / a
    tau_plus_prime = (torch.pi/2 - x + y + z) / (a + b - c)
    tau_minus_prime = (torch.pi/2 - x + y - z) / (a + b + c)
    tau2 = torch.maximum(torch.maximum(tau0_prime, tau_plus_prime), tau_minus_prime)
    # 最终结果
    return torch.minimum(tau1, tau2)


In [33]:

# 预处理数据（使用广播机制）
a_b_c = rand_abc_cache.unsqueeze(1)  # shape: (10000, 1, 3)
x_y_z = rand_su4_cache.unsqueeze(0)  # shape: (1, 100000, 3)
# 合并维度生成广播后的张量
abc = a_b_c.expand(-1, 100000, -1)   # shape: (10000, 100000, 3)
xyz = x_y_z.expand(10000, -1, -1)    # shape: (10000, 100000, 3)



In [34]:
xyz[..., 2].shape

torch.Size([10000, 100000])

In [35]:
abc[..., 0].shape

torch.Size([10000, 100000])

In [36]:

# 批量计算所有组合
all_tau = determine_gate_duration_vectorized(xyz, abc)  # shape: (10000, 100000)
# 计算每个a_b_c组合的平均持续时间
mean_durations = all_tau.mean(dim=1)  # shape: (10000)
# 计算L1范数
norms = torch.norm(rand_abc_cache, p=1, dim=1)  # shape: (10000)
# 最终结果
final_durations = mean_durations * norms
avg_duration = final_durations.mean()
print(f"Average duration: {avg_duration.item():.4f}")

KeyboardInterrupt: 